In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

/home/river/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from timeit import default_timer as timer

import numpy as np
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from numpy import argmax
from keras_helper import NNWeightHelper
from snes import SNES
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [3]:
img_width, img_height = 28, 28
train_data_dir = "/home/river/assi2_ce888/Offical31_datasets/amazon/images"

validation_data_dir = "/home/river/assi2_ce888/Offical31_datasets/dslr/images"

In [4]:
nb_train_samples = 527
nb_validation_samples = 423
epochs = 30
batch_size = 160
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


In [5]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    #save_to_dir= "/home/river/assi2_ce888/Offical31_datasets/save_pic1",
    target_size=(img_width, img_height),
    #color_mode = "grayscale",
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    #color_mode ="grayscale",
    batch_size=batch_size,
    class_mode='categorical')

Found 2817 images belonging to 31 classes.
Found 498 images belonging to 31 classes.


In [ ]:
#train_imgs, train_labels= train_generator.next()
#test_imgs, test_labels = validation_generator.next()

In [6]:
train_data = next(train_generator)

In [7]:
test_data = next(validation_generator)
test_imgs = test_data[0]
test_labels = test_data[1]

In [8]:
train_imgs = train_data[0]
train_labels = train_data[1]


In [ ]:
#train_labels = np.zeros((train1_labels.shape[0],), dtype = float)
for i in range(train1_labels.shape[0]):
    train_labels[i] = argmax(train1_labels[i])

In [ ]:
#test_labels = np.zeros((test1_labels.shape[0],), dtype = float)
for i in range(test1_labels.shape[0]):
    test_labels[i] = argmax(test1_labels[i])

In [18]:
print(train_labels.shape)

(160, 31)


In [21]:
train_generator.classes.shape

(2817,)

In [9]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(31))
model.add(Activation('softmax'))

In [10]:
model.compile(loss = "mean_squared_error",
              optimizer='adam',
              metrics=['accuracy'])

In [11]:
model.fit(train_imgs, train_labels,
          epochs=epochs,  
          batch_size=batch_size,)

Epoch 1/30
160/160 [==============================] - 1s 3ms/step - loss: 0.0312 - acc: 0.0375
Epoch 2/30
160/160 [==============================] - 0s 1ms/step - loss: 0.0313 - acc: 0.0188
Epoch 3/30
160/160 [==============================] - 0s 1ms/step - loss: 0.0312 - acc: 0.0500
Epoch 4/30
160/160 [==============================] - 0s 1ms/step - loss: 0.0312 - acc: 0.0250
Epoch 5/30
160/160 [==============================] - 0s 1ms/step - loss: 0.0312 - acc: 0.0375
Epoch 6/30
160/160 [==============================] - 0s 1ms/step - loss: 0.0312 - acc: 0.0437
Epoch 7/30
160/160 [==============================] - 0s 1ms/step - loss: 0.0312 - acc: 0.0500
Epoch 8/30
160/160 [==============================] - 0s 1ms/step - loss: 0.0312 - acc: 0.0688
Epoch 9/30
160/160 [==============================] - 0s 1ms/step - loss: 0.0312 - acc: 0.0500
Epoch 10/30
160/160 [==============================] - 0s 1ms/step - loss: 0.0312 - acc: 0.0500
Epoch 11/30
160/160 [============================

In [12]:
SAMPLE_SIZE = 1024
# how many different sets of weights ask() should return for evaluation
POPULATION_SIZE = 10
# how many times we will loop over ask()/tell()
GENERATIONS = 30

def train_classifier(model, X,y):
    #History= model.predict_generator(X, steps= 177)
    #X_features = History.history
    X_features = model.predict(X)
    #clf = ExtraTreesClassifier(n_estimators=100, n_jobs=4)
    clf = DecisionTreeClassifier()
    #clf = RandomForestClassifier(n_estimators=100, n_jobs=4)

    clf.fit(X_features,y)
    y_pred = clf.predict(X_features)
    return clf, y_pred


def predict_classifier(model, clf, X):
    #X_features = model.predict_generator(X,steps = 177)
    X_features = model.predict(X)
    return clf.predict(X_features)

In [13]:
nnw = NNWeightHelper(model)
weights = nnw.get_weights()

In [14]:
print("Total number of weights to evolve is:", weights.shape)

all_examples_indices = list(range(train_imgs.shape[0]))
clf, _ = train_classifier(model, train_imgs,train_labels)
#clf, _ = train_classifier(model, train_imgs,train_labels)
print("train_imgs shape is :", train_imgs.shape)
print("train-labels shape is :",train_labels.shape)
y_pred = predict_classifier(model, clf, test_imgs)
print(y_pred)
#y_pred = predict_classifier(model, clf, test_imgs)

print(test_labels.shape, y_pred.shape)
test_accuracy = accuracy_score(test_labels, y_pred)

print('Non-trained NN Test accuracy:', test_accuracy)

Total number of weights to evolve is: (34815,)
train_imgs shape is : (160, 28, 28, 3)
train-labels shape is : (160, 31)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(160, 31) (160, 31)
Non-trained NN Test accuracy: 0.05625


In [23]:
snes  = SNES(weights, 1 , POPULATION_SIZE)

In [24]:
for i in range(0, GENERATIONS):
    start = timer()
    asked = snes.ask()
    
    told = []
    subsample_indices  = np.random.choice(all_examples_indices, size= 15,replace = False)
    subsample_indices_valid = np.random.choice(all_examples_indices,size=15+1,replace = False)
    for asked_j in asked:
        nnw.set_weights(asked_j)
        clf, _ = train_classifier(model, train_imgs[subsample_indices], train_labels[subsample_indices])
        
        y_pred = predict_classifier(model, clf, train_imgs[subsample_indices_valid])
        score = accuracy_score(train_labels[subsample_indices_valid], y_pred)

        # clf, _ = train_classifier(model, x_train, y_train)
        # y_pred = predict_classifier(model, clf, x_test)
        # score = accuracy_score(y_test, y_pred)
        # append to array of values that are to be returned
        told.append(score)

        snes.tell(asked, told)
        end = timer()
        print("It took", end - start, "seconds to complete generation", i + 1)

nnw.set_weights(snes.center)

clf, _ = train_classifier(model, train_imgs, train_labels)
y_pred = predict_classifier(model, clf, test_imgs)
#print(told)

print(test_labels.shape, y_pred.shape)
test_accuracy = accuracy_score(test_labels, y_pred)
print('Test accuracy:', test_accuracy)

Step 1.0 : 0.0 best: 0.0 1


ValueError: shapes (1,) and (10,34815) not aligned: 1 (dim 0) != 10 (dim 0)